In [1]:
!git clone https://github.com/MockaWolke/featout_hackathon
%cd featout_hackathon
!bash load_data.sh
!pip install -e .

Cloning into 'featout_hackathon'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 87 (delta 38), reused 70 (delta 21), pack-reused 0
Receiving objects: 100% (87/87), 23.35 MiB | 27.45 MiB/s, done.
Resolving deltas: 100% (38/38), done.
/content/featout_hackathon
--2023-09-14 14:01:55--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 104.85.240.190, 2600:1409:9800:494::317f, 2600:1409:9800:48e::317f
Connecting to download.microsoft.com (download.microsoft.com)|104.85.240.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M  57.9MB/s    in 24s     

2023-09-14 14:02:20 (32.4 MB/s) - ‘kagglecatsanddogs_534

In [2]:
!git pull

Already up to date.


In [3]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from featout_exp.utils import CustomDataset, load_model
# Reading the dataframe
df = pd.read_csv('csvs/second_half.csv', index_col= 0)  # Replace 'your_data_file.csv' with your dataframe's path
df
dataset = CustomDataset(dataframe=df)

# Create DataLoader to handle batching
batch_size = 32  # You can modify this as needed
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = load_model(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    corrects = 0
    total = 0

    bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for inputs, labels in bar:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        corrects += torch.sum(preds == labels.data)
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = corrects.double() / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

print("Training complete!")


Epoch 1/10: 100%|██████████| 391/391 [01:00<00:00,  6.51it/s]


Epoch 1/10 - Loss: 0.6868 Acc: 0.5469


Epoch 2/10: 100%|██████████| 391/391 [00:51<00:00,  7.58it/s]


Epoch 2/10 - Loss: 0.6411 Acc: 0.6299


Epoch 3/10: 100%|██████████| 391/391 [00:51<00:00,  7.63it/s]


...

Epoch 9/10 - Loss: 0.3635 Acc: 0.8378


Epoch 10/10: 100%|██████████| 391/391 [00:51<00:00,  7.53it/s]


Epoch 10/10 - Loss: 0.3295 Acc: 0.8560
Training complete!


  0%|          | 0/391 [00:00<?, ?it/s]


KeyError: ignored

In [14]:
test_df = pd.read_csv('csvs/first_half.csv', index_col= 0)
test_dataset = CustomDataset(dataframe=test_df)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle= False)


model.eval()  # Set the model to evaluation mode

all_predictions = []
all_true_labels = []
all_logits = []
loss_per_image = []
criterion = nn.CrossEntropyLoss(reduction='none')  # Using 'none' to compute loss for each image


In [15]:
with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Calculate loss for each image in the batch
        losses = criterion(outputs, labels)
        loss_per_image.extend(losses.cpu().numpy())

        # Extract predicted probabilities (logits)
        probs = nn.functional.softmax(outputs, dim=1)
        all_logits.extend(probs.cpu().numpy())

        # Get predicted labels
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())

# Check the length before assigning
assert len(loss_per_image) == len(test_df)
assert len(all_predictions) == len(test_df)
assert len(all_true_labels) == len(test_df)

test_df["loss"] = loss_per_image
test_df["predicted"] = all_predictions
test_df["true_label"] = all_true_labels
test_df["logits_cat"] = [logit[0] for logit in all_logits]  # logits for class "Cat"
test_df["logits_dog"] = [logit[1] for logit in all_logits]  # logits for class "Dog"
test_df.to_csv("csvs/first_half_worst.csv")

 42%|████▏     | 165/391 [00:17<00:22,  9.95it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 391/391 [00:43<00:00,  8.95it/s]


In [16]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from featout_exp.utils import CustomDataset, load_model
# Reading the dataframe
df = pd.read_csv('csvs/first_half.csv', index_col= 0)  # Replace 'your_data_file.csv' with your dataframe's path
df
dataset = CustomDataset(dataframe=df)

# Create DataLoader to handle batching
batch_size = 32  # You can modify this as needed
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = load_model(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 10


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    corrects = 0
    total = 0

    bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for inputs, labels in bar:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        corrects += torch.sum(preds == labels.data)
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = corrects.double() / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

print("Training complete!")


Epoch 1/10:  27%|██▋       | 107/391 [00:14<00:34,  8.35it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch 1/10: 100%|██████████| 391/391 [00:53<00:00,  7.31it/s]


Epoch 1/10 - Loss: 0.6780 Acc: 0.5866


Epoch 2/10: 100%|██████████| 391/391 [00:51<00:00,  7.57it/s]


Epoch 2/10 - Loss: 0.6235 Acc: 0.6512


Epoch 3/10: 100%|██████████| 391/391 [00:52<00:00,  7.43it/s]


...

Epoch 8/10 - Loss: 0.4143 Acc: 0.8100


Epoch 9/10: 100%|██████████| 391/391 [00:52<00:00,  7.50it/s]


Epoch 9/10 - Loss: 0.3784 Acc: 0.8287


Epoch 10/10: 100%|██████████| 391/391 [00:52<00:00,  7.43it/s]

Epoch 10/10 - Loss: 0.3587 Acc: 0.8411
Training complete!


In [18]:

test_df = pd.read_csv('csvs/second_half.csv', index_col= 0)
test_dataset = CustomDataset(dataframe=test_df)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle= False)


model.eval()  # Set the model to evaluation mode

all_predictions = []
all_true_labels = []
all_logits = []
loss_per_image = []
criterion = nn.CrossEntropyLoss(reduction='none')  # Using 'none' to compute loss for each image

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Calculate loss for each image in the batch
        losses = criterion(outputs, labels)
        loss_per_image.extend(losses.cpu().numpy())

        # Extract predicted probabilities (logits)
        probs = nn.functional.softmax(outputs, dim=1)

        all_logits.extend(probs.cpu().numpy())

        # Get predicted labels
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())

# Check the length before assigning
assert len(loss_per_image) == len(test_df)
assert len(all_predictions) == len(test_df)
assert len(all_true_labels) == len(test_df)

test_df["loss"] = loss_per_image
test_df["predicted"] = all_predictions
test_df["true_label"] = all_true_labels
test_df["logits_cat"] = [logit[0] for logit in all_logits]  # logits for class "Cat"
test_df["logits_dog"] = [logit[1] for logit in all_logits]  # logits for class "Dog"
test_df.to_csv("csvs/second_half_worst.csv")

100%|██████████| 391/391 [00:41<00:00,  9.46it/s]
